# This Notebook contains all solutions from the Exercise 3

## Nr. 1

- Pick a list within the Wikipedia like the list of sovereign states. Choose some other list on your own, based on your personal interests. The only requirement is that there are other Wikipedia articles linked within the list.
## Nr.2

- Get all the names and URLs to the corresponding items in the list and export them into a CSV file that has two columns (name and URL).

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
url = "https://en.wikipedia.org/wiki/List_of_countries_with_alcohol_prohibition"
content = requests.get(url).text
soup = BeautifulSoup(content)
def titles(soup):
    present_list = soup.find_all("ul")[1]
    titles = []
    for x in present_list.find_all("a"):
        titles.append(x.get("title"))
    titles = list(set(titles))
    remove = [None, "Indonesia", "COVID-19 pandemic in South Africa", "India", "South Yemen", "Union Territory", "Sharjah (emirate)"]
    for things in remove:
        titles.remove(things)
    titles_clean = sorted(titles)
    return titles_clean
def href(titles):
    url = "https://wikipedia.org/wiki/"
    href = []
    links = []
    for country in titles:
        href.append(url)
    for i in range(len(href)):
        links.append(list(href[i] + titles[x] for x in range(len(titles))))
    links = links.pop(0)
    links_clean = []
    for entries in links:
        re = entries.replace(" ", "_")
        links_clean.append(re)
    return links_clean
def writer(title, link):
    column_names = ["Names", "URL"]
    with open("List_of_countries_with_alcohol_prohibition.csv", "w", newline = "") as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(column_names)
        writer.writerows(zip(title, link))
titles(soup)
href(titles(soup))
writer(titles(soup), href(titles(soup)))